In [1]:
import mysql.connector

## Подготовка:
Создание базы данных, таблиц

```python
mydb = mysql.connector.connect(
  host="localhost",
  user="irene",
  password="Vasya123"
)

mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE schedule")
```

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="irene",
  password="Vasya123",
  database="schedule"
)

In [3]:
mycursor = mydb.cursor()

My system's databases

In [4]:
mycursor.execute("SHOW DATABASES")

for x in mycursor:
    print(x)

('information_schema',)
('mysql',)
('performance_schema',)
('schedule',)
('sys',)


### Задание:
    У института есть преподаватели, учебные группы и аудитории. Организовать хранение расписания.
### Таблицы:
    1) BATCHES (id, Name): type(Name) = TEXT
    2) CLASSES (id, Name): type(Name) = TEXT
    3) BUILDINGS (id, Address): type(Address) = TEXT
    4) CLASSROOMS (id, Number, id Address): type(Number) = INT
    5) STUDENTS (id, Name, id Batch): type(Name) = TEXT
    6) SCHEDULE (id, Date_Time, Teacher, id Batch, Classroom, id Building, id Class, id Student): type(Date_Time) = DATETIME

In [5]:
mycursor.execute("SHOW TABLES")

for x in mycursor:
    print(x)

('BATCHES',)
('BUILDINGS',)
('CLASSES',)
('CLASSROOMS',)
('SCHEDULE',)
('STUDENTS',)


In [6]:
mycursor.execute("DROP TABLE BATCHES, CLASSES, BUILDINGS, CLASSROOMS, STUDENTS, SCHEDULE")

In [7]:
mycursor.execute("CREATE TABLE BATCHES (id INT AUTO_INCREMENT PRIMARY KEY, Name TEXT)")
mycursor.execute("CREATE TABLE CLASSES (id INT AUTO_INCREMENT PRIMARY KEY, Name TEXT)")
mycursor.execute("CREATE TABLE BUILDINGS (id INT AUTO_INCREMENT PRIMARY KEY, Address TEXT)")
mycursor.execute("CREATE TABLE CLASSROOMS (id INT AUTO_INCREMENT PRIMARY KEY, Number INT, Address INT)")
mycursor.execute("CREATE TABLE STUDENTS (id INT AUTO_INCREMENT PRIMARY KEY, Name TEXT, Batch INT)")
mycursor.execute("CREATE TABLE SCHEDULE (id INT AUTO_INCREMENT PRIMARY KEY, Date_Time DATETIME, Teacher TEXT, Batch INT, Classroom INT, Building INT, Class INT, Student INT)")

In [8]:
mycursor.execute("SHOW TABLES")

for x in mycursor:
    print(x)

('BATCHES',)
('BUILDINGS',)
('CLASSES',)
('CLASSROOMS',)
('SCHEDULE',)
('STUDENTS',)


### Что может делать юзер:
    Вписывать расписание для одного студента
    Вписывать расписание для группы

### Переменные (вводятся юзером):
    teacher_name, batch_name, class_name, building_adsress, classroom_num, student_name, date_time

In [9]:
def insert(mycursor, mydb, teacher_name, batch_name, class_name, building_address, classroom_num, student_name, date_time):
    
    '''
    Вставляем в таблицы, где нет повторов и ссылок.
    '''
    insert_without_dupl = "INSERT INTO %s (%s) SELECT * FROM (SELECT '%s') AS tmp WHERE NOT EXISTS (SELECT %s FROM %s WHERE %s = '%s')"
    sql = insert_without_dupl % ('BATCHES', 'Name', batch_name, 'Name', 'BATCHES', 'Name', batch_name)
    mycursor.execute(sql)
    sql = insert_without_dupl % ('CLASSES', 'Name', class_name, 'Name', 'CLASSES', 'Name', class_name)
    mycursor.execute(sql)
    sql = insert_without_dupl % ('BUILDINGS', 'Address', building_address, 'Address', 'BUILDINGS', 'Address', building_address)
    mycursor.execute(sql)
    
    '''
    Вставляем в таблицу, где есть повторы и ссылки (имена и номера аудиторий (в двух корпусах есть аудитория 504) могут повторяться).
    '''    
    sql = "SELECT id from BUILDINGS WHERE Address = '%s'" % building_address
    mycursor.execute(sql)
    id_address = mycursor.fetchall()[0][0]
    sql = "INSERT INTO CLASSROOMS (Number, Address) VALUES (%s, %s)" % (classroom_num, id_address)
    mycursor.execute(sql)

    # условимся, что имена внутри группы не могут повторяться
    sql = "SELECT id from BATCHES WHERE Name = '%s'" % batch_name
    mycursor.execute(sql)
    id_batch = mycursor.fetchall()[0][0]
    sql = "INSERT INTO STUDENTS (Name, Batch) SELECT * FROM (SELECT '%s', %s) AS tmp WHERE NOT EXISTS (SELECT (Name) FROM STUDENTS WHERE Name = '%s' AND Batch = %s)" % (student_name, id_batch, student_name, id_batch)
    mycursor.execute(sql)
    
    sql = "SELECT id from CLASSES WHERE Name = '%s'" % class_name
    mycursor.execute(sql)
    id_class = mycursor.fetchall()[0][0]
    sql = "SELECT id from STUDENTS WHERE Name = '%s' AND Batch = '%s'" % (student_name, id_batch)
    mycursor.execute(sql)
    id_student = mycursor.fetchall()[0][0]
    sql = "INSERT INTO SCHEDULE (Date_Time, Teacher, Batch, Classroom, Building, Class, Student) VALUES ('%s', '%s', %s, %s, %s, %s, %s)" % (date_time, teacher_name, id_batch, classroom_num, id_address, id_class, id_student)
    mycursor.execute(sql)
    mydb.commit()
    print("Inserted.")

In [10]:
teachers = ['Толдова Светлана Юрьевна', 'Нефедов Михаил Анатольевич',
            'Золотухин Всеволод Валерьевич', 'Золотухин Всеволод Валерьевич',
            'Клышинский Эдуард Станиславович', 'Литвинов Денис Владимирович']
batches = ['БКЛ-152', 'БКЛ-152', 'БКЛ-152', 'БКЛ-153', 'БКЛ-153', 'БКЛ-153']
building_address = ['Старая Басманная ул., д. 21/4', 'Кирпичная ул., д. 33',
                    'Старая Басманная ул., д. 21/4', 'Старая Басманная ул., д. 21/4',
                    'Старая Басманная ул., д. 21/4', 'Старая Басманная ул., д. 21/4']
classrooms = [505, 505, 503, 502, 321, 505]
students = ['Пантелеева Ирина Максимовна', 'Пантелеева Ирина Максимовна',
            'Пантелеева Ирина Максимовна', 'Глазунов Евгений Владимирович',
            'Глазунов Евгений Владимирович', 'Глазунов Евгений Владимирович']
date_time = ['2018-02-11 09:00:00', '2018-02-11 10:30:00',
             '2018-02-11 12:10:00', '2018-02-11 09:00:00',
             '2018-02-11 10:30:00', '2018-02-11 12:10:00',]
classes = ['Компьютерная лингвистика и информационные технологии',
           'Компьютерная лингвистика и информационные технологии', 'Философия',
           'Философия', 'Базы данных', 'Программирование']

In [11]:
for x in range(6):
    print('---')
    print(students[x], '\n', classes[x], '\n', teachers[x], '\n', batches[x],
          '\n', building_address[x], '\n', classrooms[x], '\n',
          date_time[x], '\n',)
    insert(mycursor, mydb,
           teacher_name=teachers[x],
           batch_name=batches[x],
           class_name=classes[x],
           building_address=building_address[x],
           classroom_num=classrooms[x],
           student_name=students[x],
           date_time=date_time[x])

---
Пантелеева Ирина Максимовна 
 Компьютерная лингвистика и информационные технологии 
 Толдова Светлана Юрьевна 
 БКЛ-152 
 Старая Басманная ул., д. 21/4 
 505 
 2018-02-11 09:00:00 

Inserted.
---
Пантелеева Ирина Максимовна 
 Компьютерная лингвистика и информационные технологии 
 Нефедов Михаил Анатольевич 
 БКЛ-152 
 Кирпичная ул., д. 33 
 505 
 2018-02-11 10:30:00 

Inserted.
---
Пантелеева Ирина Максимовна 
 Философия 
 Золотухин Всеволод Валерьевич 
 БКЛ-152 
 Старая Басманная ул., д. 21/4 
 503 
 2018-02-11 12:10:00 

Inserted.
---
Глазунов Евгений Владимирович 
 Философия 
 Золотухин Всеволод Валерьевич 
 БКЛ-153 
 Старая Басманная ул., д. 21/4 
 502 
 2018-02-11 09:00:00 

Inserted.
---
Глазунов Евгений Владимирович 
 Базы данных 
 Клышинский Эдуард Станиславович 
 БКЛ-153 
 Старая Басманная ул., д. 21/4 
 321 
 2018-02-11 10:30:00 

Inserted.
---
Глазунов Евгений Владимирович 
 Программирование 
 Литвинов Денис Владимирович 
 БКЛ-153 
 Старая Басманная ул., д. 21/4 
 505 
 

Посмотрим как всё записалось:

In [12]:
tables = ['BATCHES', 'CLASSES', 'BUILDINGS', 'CLASSROOMS', 'STUDENTS', 'SCHEDULE']

In [13]:
for table in tables:
    mycursor.execute("SELECT * FROM %s" % table)
    myresult = mycursor.fetchall()
    print('---')
    print('Table:', table)
    for x in myresult:
        print(x)

---
Table: BATCHES
(1, 'БКЛ-152')
(2, 'БКЛ-153')
---
Table: CLASSES
(1, 'Компьютерная лингвистика и информационные технологии')
(2, 'Философия')
(3, 'Базы данных')
(4, 'Программирование')
---
Table: BUILDINGS
(1, 'Старая Басманная ул., д. 21/4')
(2, 'Кирпичная ул., д. 33')
---
Table: CLASSROOMS
(1, 505, 1)
(2, 505, 2)
(3, 503, 1)
(4, 502, 1)
(5, 321, 1)
(6, 505, 1)
---
Table: STUDENTS
(1, 'Пантелеева Ирина Максимовна', 1)
(2, 'Глазунов Евгений Владимирович', 2)
---
Table: SCHEDULE
(1, datetime.datetime(2018, 2, 11, 9, 0), 'Толдова Светлана Юрьевна', 1, 505, 1, 1, 1)
(2, datetime.datetime(2018, 2, 11, 10, 30), 'Нефедов Михаил Анатольевич', 1, 505, 2, 1, 1)
(3, datetime.datetime(2018, 2, 11, 12, 10), 'Золотухин Всеволод Валерьевич', 1, 503, 1, 2, 1)
(4, datetime.datetime(2018, 2, 11, 9, 0), 'Золотухин Всеволод Валерьевич', 2, 502, 1, 2, 2)
(5, datetime.datetime(2018, 2, 11, 10, 30), 'Клышинский Эдуард Станиславович', 2, 321, 1, 3, 2)
(6, datetime.datetime(2018, 2, 11, 12, 10), 'Литвинов 

### Запросы
    Задание: Вы должны показать владение операторами SELECT и прочими, показав что вы умеете делать JOIN, или искать пересечение таблиц в WHERE, или пользоваться вложенными SELECT.
    
    1) Показать расписание определённого студента
    2) Показать загруженность аудитории на определённый день

#### 1) Показать расписание определённого студента

In [86]:
def schedule_of_student(student_name, batch_name):
    sql = "SELECT SCHEDULE.Date_Time, SCHEDULE.Teacher, SCHEDULE.Classroom, BUILDINGS.Address, STUDENTS.Name, CLASSES.Name, BATCHES.Name FROM SCHEDULE JOIN BUILDINGS ON SCHEDULE.Building = BUILDINGS.id JOIN CLASSES ON SCHEDULE.Class = CLASSES.id JOIN BATCHES ON SCHEDULE.Batch = BATCHES.id JOIN STUDENTS ON SCHEDULE.Student = STUDENTS.id WHERE STUDENTS.Name = '%s' AND BATCHES.Name = '%s'" % (student_name, batch_name)
    mycursor.execute(sql)
    id_student = mycursor.fetchall()
    return id_student

In [87]:
schedule_of_student('Пантелеева Ирина Максимовна', 'БКЛ-152')

[(datetime.datetime(2018, 2, 11, 9, 0),
  'Толдова Светлана Юрьевна',
  505,
  'Старая Басманная ул., д. 21/4',
  'Пантелеева Ирина Максимовна',
  'Компьютерная лингвистика и информационные технологии',
  'БКЛ-152'),
 (datetime.datetime(2018, 2, 11, 10, 30),
  'Нефедов Михаил Анатольевич',
  505,
  'Кирпичная ул., д. 33',
  'Пантелеева Ирина Максимовна',
  'Компьютерная лингвистика и информационные технологии',
  'БКЛ-152'),
 (datetime.datetime(2018, 2, 11, 12, 10),
  'Золотухин Всеволод Валерьевич',
  503,
  'Старая Басманная ул., д. 21/4',
  'Пантелеева Ирина Максимовна',
  'Философия',
  'БКЛ-152')]